In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
from matplotlib.patches import Rectangle
sns.set()

# Introduction <a id='intro'></a>

This notebook uses a variety of different data sets to investigate statistical significance
between quantities; the main result is the application of two-way ANOVA to find the
relationship between different types of quarantine measures and countries and the COVID growth rate.



# Table of contents

## [Function definitions](#functions)

## [Data](#imports)

## [Summary Statistics](#stats)

## [Testing the importance of different quarantine measures with two-way ANOVA](#ANOVA2)


## Functions  <a id='functions'></a>

In [1]:
def quarantine_plot(data, country, response_data, feature='cases_per_1M_people', window='all_active_dates', applyfunc=None):
    responses_start = response_data.loc[country, :].dropna().iloc[::2].sort_index()
    responses_end = response_data.loc[country, :].dropna().iloc[1::2].sort_index()
    # reformat the index (quarantine measure names) for plotting purposes
    responses_start.index = responses_start.index.str.split('_').str.slice(stop=-1).str.join(sep=' ')
    responses_end.index = responses_end.index.str.split('_').str.slice(stop=-1).str.join(sep=' ')
    
    # The indices are the same to make subtraction well defined; reorder after subtraction to make chronological.
    if window == 'all_active_dates':
        window_widths = (responses_end-responses_start).loc[responses_start.sort_values().index]
        left_title = 'All active dates'
    elif window == 'until_next_measure':
        window_widths = np.abs(responses_start.sort_values().diff(-1))
        window_widths.iloc[-1] = pd.Timedelta(seconds=0)
        left_title = 'Until next measure'
    cmap = plt.get_cmap('plasma')
    my_colors = [cmap(int(i)) for i in np.linspace(cmap.N-32, 32, len(responses_start))]
    fig, (ax, ax2) = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
    if applyfunc is None:
        series_to_plot = data.loc[country, feature]
    else:
        series_to_plot = data.loc[country, feature].apply(applyfunc) 
    series_to_plot.plot(ax=ax,color='k',label='')
    series_to_plot.plot(ax=ax2,color='k',label='')

    ymin, ymax = ax.get_ylim()
    # for loop is ugly but I didn't want to spend time attempting to vector matplotlib. 
    for i, (df_index, df_val) in enumerate(responses_start.sort_values().iteritems()):

        ax.vlines(df_val, ymin=ymin, ymax=ymax, label=df_index, color=my_colors[i])
        ax.add_patch(Rectangle((responses_start.loc[df_index], 0),
                                window_widths.loc[df_index], ymax, angle=0.0, color=my_colors[i], alpha=0.5))
        ax2.vlines(df_val, ymin=ymin, ymax=ymax, label=df_index, color=my_colors[i])
        ax2.add_patch(Rectangle((responses_start.loc[df_index], 0),
                                pd.Timedelta(days=14), ymax, angle=0.0, color=my_colors[i], alpha=0.5))
            

    _ = ax.set_ylim([0, 0.98*ymax])
    _ = ax.set_title(left_title)
    _ = ax.legend(loc=(2.25, 0), title='Government Reponse')
    _ = ax.set_xlabel('')
    _ = ax.set_ylabel('$\\log(\\frac{Total\:cases}{1M\,people})$', fontsize=18)
    _ = ax2.set_xlabel('')
    _ = ax2.set_title('14-day window')
    plt.show()
    return None

def multi_index_slice_to_average_rate_original(x, testing_df):
    sliced_values = testing_df.loc[x, :]
    sliced_values_of_interest = sliced_values.loc[:, ['total_cases_per_million', 'total_tests']].replace(to_replace=np.nan, value=1.) 
#     avg_case_rate = (sliced_values_of_interest.iloc[-1, 0]
#                               -sliced_values_of_interest.iloc[0, 0])  / len(sliced_values_of_interest)
    avg_case_rate = (sliced_values_of_interest.iloc[-1, 0]/sliced_values_of_interest.iloc[0, 0])**(-1/len(sliced_values_of_interest))
#         test_weight = (sliced_values_of_interest.iloc[-1, 1]
#                        -sliced_values_of_interest.iloc[0, 1])
#         if (test_weight == 0.) or (test_weight == np.nan):
#             test_weight = 1.
#         weighted_avg_rate = avg_case_rate / test_weight
#         return weighted_avg_rate
    return avg_case_rate

def multi_index_slice_to_average_rate(x):
    sliced_values = data.loc[x, :]
    sliced_values_of_interest = sliced_values.loc[:, 'cases_per_1M_people_per_100k_tests']

    # x is a tuple of the form ('country', slice(A,B))
    if len(sliced_values_of_interest) == 0:
        return np.nan
    if x[1].start is None:
        before_df = sliced_values_of_interest.iloc[:-1]
        cutoff = sliced_values_of_interest.iloc[-1]
        avg_case_rate = cutoff-before_df.mean()
    else:
        after_df = sliced_values_of_interest.iloc[1:]
        cutoff = sliced_values_of_interest.iloc[0]
        avg_case_rate = after_df.mean()-cutoff
     
    return avg_case_rate

def multiindex_response_date_to_average_rates(x):
    country = x[0]
    date = x[1]
    before_slice = pd.IndexSlice[date-pd.Timedelta(days=14):date-pd.Timedelta(days=1)]
    after_slice =  pd.IndexSlice[date+pd.Timedelta(days=1):date+pd.Timedelta(days=14)]
    cutoff = data.loc[(country, date), 'cases_per_1M_people']
    before_df = data.loc[(country, before_slice), 'cases_per_1M_people']
    after_df = data.loc[(country, after_slice), 'cases_per_1M_people']
    # If windows are the same width, do not need 1/dT factor.
    avg_rate_before = np.log(cutoff/before_df.mean())
    avg_rate_after = np.log(after_df.mean()/cutoff)

    return avg_rate_after - avg_rate_before


## Data  <a id='data'></a>

## Summary Statistics  <a id='stats'></a>

## Is the United States' situation statistically different from other countries?  <a id='ANOVA2'></a>

## Testing the importance of different quarantine measures with two-way ANOVA  <a id='ANOVA2'></a>

to be able to use apply later, zip the slices with the country names so that they look like multiindex slices
Can't do ANOVA on missing values, drop them; this has ramifications in the future of filtering countries.
There are a number of countries with mismatched labels (different data sources), correct these manually. 
The original numbers are in units of thousands of people
Because I have three different data sources for the population data, government response data, and testing data,
I first take the intersection of the countries which exist in all three data sets.

Three datasets were used in the test of the effects of country and government mandates. These three datasets included
data on the dates of different quarantine measures per country \cite{govtresponse},  the time series data for the case numbers \cite{owid} and the time series data on the number of tests \cite{find}. These datasets are quite inconsistent due to differences in reaction to the pandemic and the inconsistency of reporting. This manifests as irregular time-series, in terms of the dates on which they are defined, as well as missing values.
Because of this, a number of actions which may or may not be heavy handed needed to be employed.
These actions accounted for the following: differences in time series ranges, missing values, quarantine measures which occur before the first recorded case in a country, dissimilar quarantine measures taken and differences in the countries whose data was recorded.
To make the data uniform, the time series were normalized to be from December 31st 2019 to April 20th, 2020. The end date was simply because one of the data sets has not been updated in the past week. The original missing values and those created by this normalization were handled by using linear interpolation on the interior of the time series and linear extrapolation for the beginning of the time series. This is only applied at the very beginning of each countries time series (if values are missing) typically when the cases number is very small or even equal to 1. Therefore, I believe this action is justified by using linear expansion of exponential growth. After the time series were made uniform, the specific government responses to investigate
were chosen by how widespread their adoption was. The countries were determined by taking the intersection of all countries present in the three data sets. 

To do list:

Filter government response data to find when each measure was enacted for each country.
Get population and testing data for weights
Calculate average growth rates before and after each government response.
This is done by calculating the rates using three data points for each quarantine measure.
The date of first case (subscript $i$), the date of the mandate (subscript $m$), 
and the most recent date or date that the response ended (subscript $f$).
On these two different intervals, calculate the rates before and after which are given by
    
\begin{align}
r_{\text{before}} &= \frac{1}{\Delta t_{mi}}\log(\frac{\phi_m}{\phi_i}) \\
r_{\text{after}} &= \frac{1}{\Delta t_{fm}}\log(\frac{\phi_f}{\phi_m})\,,
\end{align}

where $\phi_j$ is the number of cases per population per tests on a given day, $t = t_j$ 

4. TWO_WAY ANOVA where countries are FACTOR A, response type is FACTOR B and BEFORE AFTER are the N's.

To perform these calculations, need the dates which define the before and after intervals (unique to each country
and response type pair, from https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker) and need the cases per population per tests. The cases per population is
given in one of the data sets (original testing data, has too many missing values) https://ourworldindata.org/covid-testing . The (more reliable) number of tests is given in the dataset acquired at https://finddx.shinyapps.io/FIND_Cov_19_Tracker/

The property that makes this process very tricky is that all three datasets have variable time series: that is, they vary between datasets, between countries and even between the dates themselves. What I mean by the last part is that sometimes accurate testing numbers were not recorded and so there are either missing values or horizontal testing curves for whatever period of time. My choice for how to handle this difficulty is to use pandas DataFrames, specifically which a multi level index where the levels correspond to the countries and then the dates of the time series. 

Import the .csv file that contains information on government responses and action events.


Two-way ANOVA: Factor A will be quarantine type? Factor B is Country? Only count countries where
quarantine has been active for 2 weeks consectutively. The trials will be the time series. (weighted by population, and number of tests).

Take a subset of data which has implemented all quarantine measures that will be tested. 
The quantifiable measure will be 


OK: averaging over the time series is dubious because of confounding factors. 

Description of raw data

Three datasets were used in the test of the effects of country and government mandates. These three datasets included
data on [] the dates of different quarantine measures per country, [] the time series data for the case numbers and []
the time series data on the number of tests. These datasets are quite inconsistent due to differences in reaction to the pandemic and the inconsistency of reporting. This manifests as irregular time-series, in terms of the dates on which they are defined, as well as missing values. 

Because of this, a number of actions which may be heavy handed needed to be employed.  
These accounted for the following: differences in time series ranges, missing values, quarantine measures which occur before the first recorded case in a country, dissimilar quarantine measures and country data. 
I elected to attempt to make the data as uniform as possible; that is, by using interpolation
for missing values on the interior of the time series and linear extrapolation for the beginning of the time series. The rationale for using a linear extrapolation procedure is thus: I am not extrapolating the "true" number of COVID-19 cases, only
the ones confirmed by testing. In other words I think that this is amenable because of the deployment of testing; the true number of cases may be growing exponentially but I am assuming that the growth in confirmed cases is small at the very onset of the pandemic. 

0. determine which government responses to use, based on how widespread their adoption was. 
['School closing',
 'Workplace closing',
 'Cancel public events',
 'Restrictions on internal movement',
 'International travel controls']

1. Take the intersection of all datasets countries

2. Make the time series uniform

3. Incorporate # tests and population.

4. Decide on how to average, use the response dates. 

Because the data cleaning and wrangling took forever I have no time to consider other methods, even though

Slice out all "notes" and targeted vs. general response columns. This currently goes beyond what I want to do. Also
merge day, month, year data into single date timestamp

Because each computation requires its own unique slice, with a multiindex no less, I find it easiest
to create a DataFrame whose values are pandas multislice elements using pandas IndexSlice objects. 
These values cannot be passed at once to the testing multiindex array, but it was designed to take advantage of the .apply method. 

In order to incorporate the largest segment of the population, but remove all missing values, drop the
public transport columns and then drop the remaining countries with missing values.  

Just using the endpoints of each interval is not going to work as well, because if the endpoints represent outliers then they
will not capture the overall trend. Therefore, I will do the following: average the two intervals before and after the quarantine measure (average the cases/((1M people)(100k tests)) and then compare the averages with the value at the quarantine date. I believe this is fair because it's being applied equally to both intervals.

With the world currently under maintenance and public sentiment deteriorating, it is important to identify
the most efficient quarantine measures so that governments know how to be proactive going forward.
Given a set of countries how do specific quarantine measures affect the growth rate of COVID-19? To analyze this, we used the strictly increasing quantity of total number of tests per capita. Th main external factor which was not taken into account is the fact that the more tests a country performs the more cases typically emerge; i.e., the rate over time
may increase as a result of increased testing and not spread of the disease itself.

Figure \ref{fig:SKrate} demonstrates a time series of total cases for South Korea. The widths
of the colored regions represent different quantities; on the left it demonstrates the total amount
of time certain measures have been in effect while on the right it only shows a 14-day window
after the implementation of each measure. These 14-day windows attempt to account for the time-delayed effect resulting from incubation time.
The claim that we are pushing is that the rate of change increased until 14 days after the strictest level of quarantine measures were introduced, implying that only then were the measures sufficient.
This behavior is incorporated directly into the analysis. For every quarantine measure, 14 day windows of the time series are taken prior and after each implementation date. The average of the total cases per capita is taken on these windows which results in a triplet of past, present, and future cases per capita values. By assuming exponential growth,
the growth rate before the quarantine measure and after as well as their difference are calculated using
\begin{align}
\rho_{i} &= \frac{1}{\Delta t_{mi}}\log(\frac{\phi_m}{\bar{\phi}_i})\nonumber \\
\rho_{f} &= \frac{1}{\Delta t_{fm}}\log(\frac{\bar{\phi}_{f}}{\phi_m})\nonumber \\
\rho &= \rho_{f}-\rho_{i} \,,
\label{eq:avgrates}
\end{align}
where over-bars denote averages over each window.
By iterating over the countries and government responses, equation (\ref{eq:avgrates}) produces a table of observations on which two-way ANOVA can be applied. Specifically,
the blocks were taken to be the different quarantine measures and the treatments were taken to be the countries. The confidence level is chosen to be 95\% and the corresponding hypothesis set can be written as
\begin{align}
    H_0 &: \mu_0 = \mu_1 = ... = \mu_a \nonumber\\
    H_1 &: \text{any of the means is different} \nonumber
\end{align}

The testing data is being used as a reference, so its not necessary to remove the countries which aren't
in the government response data.

Unfortunately there is one more step that needs to be taken, as even though the testing data has data on
some countries, it doesn't contain the actual testing data. Therefore, need to remove countries which do not
have testing data.

Countries where there is actual testing data and case data. 

Only look at the average growth rates after first known case, as to not unfairly decrease the rate before responses.


These represent the official dates of government action but these dates may not exist in testing data. Therefore, need to check start dates vs. testing data start dates.

testing data typically starts far after response data. therefore cannot weight by testing and still have representation from majority of countries. In that case, just use the average rate of total cases per million, before and after response dates.

Calculate the extrapolation range by comparing first known case date with first reported test date (obviously
there must to have been tests before this date, they just weren't recorded). More concerned with getting the extrapolation right, so don't worry if it doesn't match where there is actual testing data.

from scipy.stats import kstest

_ = plt.hist(before_minus_after.values.ravel(),bins=20)
_ = plt.xlabel('Case-test ratio growth rate')
_ = plt.ylabel('Frequency')
_ =  plt.savefig('mg2.jpg',bbox_inches='tight', pad_inches=0)
_ = plt.show()
print(kstest(before_minus_after.values.ravel(), cdf='norm'))

In [ ]:
# old version
# all_responses = group_by_country.iloc[:, [0, 1, 2, 3, 5, 6]]
# country_list = []
# slice_list = []

# for j, (country, country_df) in enumerate(all_responses.groupby(level=0)):
#     active_dates = country_df.replace(to_replace=0., value=np.nan)
#     country_list += [country]
#     before_list = []
#     after_list = []
#     for i, single_response in enumerate(active_dates.columns):
#         effective_range = active_dates[single_response].dropna(axis=0)
#         before = effective_range.reset_index().Date.min()
#         after = effective_range.reset_index().Date.max()
#         slice_list += [pd.IndexSlice[:before], pd.IndexSlice[before:after]]   
        
# before_after_columns = np.array([[x+'_before', x+'_after'] for x in all_responses.columns.tolist()]).ravel()
# enacted_ended_df = pd.DataFrame(np.array(slice_list).reshape(len(country_list), -1), index=country_list, columns=before_after_columns)
# enacted_ended_df_multiindex_final = enacted_ended_filtered_final_df.copy()
# for c in enacted_ended_filtered_final_df.columns:
#     enacted_ended_df_multiindex_final.loc[:, c] = list(zip(enacted_ended_df_multiindex_final.index.tolist(), 
#                                                enacted_ended_df_multiindex_final.loc[:, c].tolist()))
all_responses = response_df.iloc[:, 1:17:2]

country_list = []
slice_list = []

for j, (country, country_df) in enumerate(all_responses.groupby(level=0)):
    active_dates = country_df.replace(to_replace=0., value=np.nan)
    country_list += [country]
    before_list = []
    after_list = []
    for i, single_response in enumerate(active_dates.columns):
        effective_range = active_dates[single_response].dropna(axis=0)
        before = effective_range.reset_index().Date.min()
#         after = effective_range.reset_index().Date.max()
        slice_list += [before]   
        
enacted_ended_df = pd.DataFrame(np.array(slice_list).reshape(len(country_list), -1), index=country_list, columns=all_responses.columns)

all_responses = response_df.iloc[:, [0, 1, 2, 3, 5, 6]]
country_list = []
minmax_list = []
for j, (country, country_df) in enumerate(all_responses.groupby(level=0)):
    active_dates = country_df.replace(to_replace=0., value=np.nan)
    country_list += [country]
    for i, single_response in enumerate(active_dates.columns):
        effective_range = active_dates[single_response].dropna(axis=0)
        before = effective_range.reset_index().Date.min()
        after = effective_range.reset_index().Date.max()
        minmax_list += [before, after]   

start_end_columns = np.array([[x+'_start', x+'_end'] for x in all_responses.columns.tolist()]).ravel()
start_end_df = pd.DataFrame(np.array(minmax_list).reshape(len(country_list), -1), index=country_list, columns=start_end_columns)
start_end_filtered_df = start_end_df.drop(columns=['Close_public_transport_start','Close_public_transport_end']).dropna(axis=0)
filtered_countries = start_end_filtered_df.index
enacted_ended_filtered_df = enacted_ended_df.drop(columns=['Close_public_transport']).loc[filtered_countries, :]
start_end_filtered_df = start_end_df.drop(columns=['Close_public_transport_start','Close_public_transport_end']).dropna(axis=0)
filtered_countries = start_end_filtered_df.index
enacted_ended_filtered_df = enacted_ended_df.drop(columns=['Close_public_transport']).loc[filtered_countries, :]

In [ ]:
_ = plt.hist(before_minus_after_residual_df.values.ravel(), bins=20)
_ = plt.xlabel('Residual')
_ = plt.ylabel('Frequency')
plt.savefig('mg3.jpg',bbox_inches='tight', pad_inches=0)
_ = plt.show()
kstest(before_minus_after_residual_df.values.ravel(), cdf='norm')

Variance comparisons for treatment residuals (left, countries) and block residuals (right, government responses). There are too many countries to label conveniently but the order from left to right
  is equivalent to the lexicographical ordering in the corresponding
  appendix table
  
To validate these results the distribution of the observations and their residuals (Figure \ref{TCHist}) as well as variance comparison plots for countries and government responses are displayed (Figure \ref{TCvarbar}). The residuals and observations are approximately normally distributed and the variances for the blocks seem similar. The main area of concern is the variances of each countries' residuals. These disparate variances affect the interpretation of the ANOVA results; especially the effect that the countries have on the rate change. Additionally, these results may be misleading because the quarantine measures overlap with one another; when combined with the time delayed nature of the problem these results should be viewed skeptically.

In [ ]:
fig, ax = plt.subplots()
for i, x in enumerate(before_minus_after_residual_df.values):
    ax.plot(5*[i], x, marker='.', label=before_minus_after_residual_df.index[i])
    
_ = ax.set_ylabel('Residual')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.savefig('mg4.jpg',bbox_inches='tight', pad_inches=0)
_ = plt.show()

In [ ]:
sm_df = before_minus_after.unstack().reset_index()
sm_df.columns = ['Type','Country','Change']

import statsmodels.api as sm

formula = 'Change ~ Type + Country'
model = sm.formula.ols(formula, data=sm_df).fit()

# Perform ANOVA and print table
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)